# Expected Photon Count per Detector

In [12]:
from math import *

STATES = ['H', 'V', 'D', 'A']

STATES_MAP = {
    'H': (1, 0),
    'V': (0, 1),
    'D': (1/1/sqrt(2), 1/1/sqrt(2)),
    'A': (1/1/sqrt(2), -1/1/sqrt(2)),
}


Ne = lambda ah, av, bh, bv: (av - bv)**2 / 2
Nf = lambda ah, av, bh, bv: (ah - bh)**2 / 2
Ng = lambda ah, av, bh, bv: (av + bv)**2 / 2
Nh = lambda ah, av, bh, bv: (ah + bh)**2 / 2


outputs = {
    "Alice": [],
    "Bob": [],
    "Ne": [],
    "Nf": [],
    "Ng": [],
    "Nh": []
}

for alice in STATES:
    for bob in STATES:

        ah, av = STATES_MAP[alice]
        bh, bv = STATES_MAP[bob]

        # Divide by 2 to get probability of click in detector (2 because 2 total input photons)
        ne = round(Ne(ah, av, bh, bv) / 2, 3) * 100
        nf = round(Nf(ah, av, bh, bv) / 2, 3) * 100
        ng = round(Ng(ah, av, bh, bv) / 2, 3) * 100
        nh = round(Nh(ah, av, bh, bv) / 2, 3) * 100

        total_clicks = sum([ne, nf, ng, nh])

        outputs["Alice"].append(alice)
        outputs["Bob"].append(bob)

        outputs["Ne"].append(ne)
        outputs["Nf"].append(nf)
        outputs["Ng"].append(ng)
        outputs["Nh"].append(nh)

In [13]:
import pandas as pd

df = pd.DataFrame(outputs)  
df = df.set_index(["Alice", "Bob"])  

df

Ne    Nf     Ng     Nh
Alice Bob                          
H     H     0.0   0.0    0.0  100.0
      V    25.0  25.0   25.0   25.0
      D    12.5   2.1   12.5   72.9
      A    12.5   2.1   12.5   72.9
V     H    25.0  25.0   25.0   25.0
      V     0.0   0.0  100.0    0.0
      D     2.1  12.5   72.9   12.5
      A    72.9  12.5    2.1   12.5
D     H    12.5   2.1   12.5   72.9
      V     2.1  12.5   72.9   12.5
      D     0.0   0.0   50.0   50.0
      A    50.0   0.0    0.0   50.0
A     H    12.5   2.1   12.5   72.9
      V    72.9  12.5    2.1   12.5
      D    50.0   0.0    0.0   50.0
      A     0.0   0.0   50.0   50.0